# Experimenting Optical Flow

In [1]:
''' use base anaconda kernel '''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
from PIL import Image

import time

from pathlib import Path

import cv2 as cv

import torch
import torchvision 

/home/ubuntu/anaconda3/lib/python3.7/site-packages/setuptools/_importlib.py:42: SetuptoolsWarning: Incompatibility problem.
!!

        ********************************************************************************
        `importlib-metadata` version is incompatible with `setuptools`.
        This problem is likely to be solved by installing an updated version of
        `importlib-metadata`.

        See https://github.com/python/importlib_metadata/issues/396 for details.
        ********************************************************************************

!!
  disable_importlib_metadata_finder(metadata)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# load in the dataframe:

df_cleaned = pd.read_csv('df_cleaned.csv')

df_cleaned.head()


,name,videoName,frameIndex,id,category,attributes.crowd,attributes.occluded,attributes.truncated,box2d.x1,box2d.x2,box2d.y1,box2d.y2,haveVideo
0,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89537.0,car,False,True,False,825.173210,1003.094688,355.011547,418.198614,True
1,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89538.0,car,False,True,False,484.295612,700.461894,346.697460,424.849885,True
2,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89539.0,pedestrian,False,True,False,645.588915,663.879908,338.383372,358.337182,True
3,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89540.0,car,False,False,False,120.969977,192.471132,359.168591,409.053118,True
4,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89541.0,car,False,False,False,251.501155,315.519630,354.180139,400.739030,True


In [3]:
# initialize an example video to do this on:

video = df_cleaned['videoName'].unique()[0]
print(video)

frameNums = df_cleaned[df_cleaned['videoName'] == video]['frameIndex'].unique()
print("Number of frames in the video: ", df_cleaned[df_cleaned['videoName'] == video]['frameIndex'].nunique())

print(frameNums)

01c71072-718028b8
Number of frames in the video:  202
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201]


In [4]:

df_cleaned[df_cleaned['videoName'] == video].head()

,name,videoName,frameIndex,id,category,attributes.crowd,attributes.occluded,attributes.truncated,box2d.x1,box2d.x2,box2d.y1,box2d.y2,haveVideo
0,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89537.0,car,False,True,False,825.173210,1003.094688,355.011547,418.198614,True
1,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89538.0,car,False,True,False,484.295612,700.461894,346.697460,424.849885,True
2,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89539.0,pedestrian,False,True,False,645.588915,663.879908,338.383372,358.337182,True
3,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89540.0,car,False,False,False,120.969977,192.471132,359.168591,409.053118,True
4,01c71072-718028b8-0000001.jpg,01c71072-718028b8,0,89541.0,car,False,False,False,251.501155,315.519630,354.180139,400.739030,True


In [5]:
# picking an example video:

print(f"Number of videos: {df_cleaned['videoName'].nunique()}")


video = df_cleaned['videoName'].unique()[0]
print(f"Video Name: {video}")

frameNums = df_cleaned[df_cleaned['videoName'] == video]['frameIndex'].unique()
numFrames = len(frameNums)

print(f"Number of frames for selected video: {numFrames}")

Number of videos: 961
Video Name: 01c71072-718028b8
Number of frames for selected video: 202


In [6]:
# CHECKING BIT DEPTH OF AN IMAGE

# Load the image with the flag IMREAD_ANYDEPTH
image = cv.imread('frames/01c71072-718028b8-0000001.jpg', cv.IMREAD_ANYDEPTH)

# Check the depth of the image
depth = image.dtype

print(f"Current Bit Depth: {depth}")


Current Bit Depth: uint8


In [7]:
'''
CONNECT TO GPU OF AWS
'''

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    print("GPU available.... connecting now")
    print(torch.cuda.get_device_name(0))  # Print GPU name
else:
    print("GPU not available")

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




GPU available.... connecting now
NVIDIA A10G


In [10]:
""""
cv.calcOpticalFlowPyrLK(prevImg, nextImg, prevPts, nextPts[, status[, err[, winSize[, maxLevel[, criteria[, flags[, minEigThreshold]]]]]]])

    - prevImg --> first/previous image
    - nextImg --> 2nd input image of same size and type as prevImg
    - prevPts --> vector of 2D points for which the flow needs to be found (coordinates must be single-precision floating point numbers)
    - nextPts --> output vector of 2D points containing the calculated new positions of input features in the 2nd image
              --> when OPTFLOW_SUE_INITIAL_FLOW flag is passed, the vector must have the same size as in the input
    - status --> 1 if the flow of the corresponding features has been found; 0 otherwise
    - err --> output vector of errors; each element of the vector is set to an error for the corresponding feature 
          --> if the flow hasn't been found, then no error defined
    - winSize --> size of the search windowat each pyramid level
    - maxLevel --> ...
    - ... 
"""

###############################
print('\DEFINING PARAMETERS\n')
###############################

# calculate sparse optical flow
lk_params = dict(winSize = (15,15),
                 maxLevel = 2,
                 criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

'''
maxCorners - max numberof corners to track
qualityLevel - will discard corners below this quality level (only tracking good features/corners)
minDistance - minimum distance between corners
'''

###############################
print('\DEFINING FEATURE PARAMETERS\n')
###############################

feature_params = dict(maxCorners = 20,
                      qualityLevel = 0.3,
                      minDistance = 10,
                      blockSize = 7)

'''
trajectory_len - number points in each of the trajectories for each of our points
detect_interval - how often we want to update our corners we want to use to create this sparse feature set (using the parameters defined above)
trajectories - list of all trajectories of the points being tracked around the image frame
frame_idx - initialize frame index
'''

###############################
print('\INITIALIZING VARIABLES FOR TRAJECTORIES\n')
###############################

trajectory_len = 20
detect_interval = 1 # updates every time it runs thru the algorithm 
trajectories = []
frame_idx = 0


# capture = cv.VideoCapture(0)

# define filepath
path = 'frames/'

###############################
print('\nBEFORE THE WHILE LOOP\n')
###############################

'''
perform this only for the first 10% of the frames of the video to see if it works
'''
while frame_idx < numFrames:

    # used to give the more time for certain processes to complete 
    #################
    time.sleep(5)
    #################

    ###############################
    print('\nSUCCESSFULLY ENTERED WHILE LOOP\n')
    ###############################

    # timer to calculate frames per second
    start = time.time()

    frame = frame_idx + 1


    if frame < 10:
        file_name = f'{video}-000000{frame}.jpg'
        print(file_name)
    elif frame < 100 and frame >= 10:
        file_name = f'{video}-00000{frame}.jpg'
        print(file_name)
    elif frame < 1000 and frame >= 100:
        file_name = f'{video}-0000{frame}.jpg'
        print(file_name)

    # # used to give the more time for certain processes to complete 
    # #################
    # time.sleep(5)
    # #################

    # load the image in:
    img = cv.imread(path + file_name)

    # REDUCE MEMORY USAGE:

    # # Convert to a palette with 16 colors (4-bit)
    # reduced_bit_depth_image = img.convert('P', palette=Image.ADAPTIVE, colors=16)
    # # Simulate compression and save compressed image to variable
    # compressed_image_data = BytesIO()
    # reduced_bit_depth_image.save(compressed_image_data, format='JPEG', quality=80)  # Adjust quality as needed


    # # used to give the more time for certain processes to complete 
    # #################
    # time.sleep(5)
    # #################


    # gray scale frame:
    frame_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # original colored frame:
    frame_color = img.copy()

    # # used to give the more time for certain processes to complete 
    # #################
    # time.sleep(5)
    # #################

    ###############################
    # print("image:")
    # cv.imshow("LOADED IMAGE:", frame_gray)
    ###############################


    # calculate optical flow for a sparse feature set using the iterative Lucas-Kande method

    # if there are any obejects tracked or already being trakced around the image frame
    if len(trajectories) > 0:
        img0, img1 = prev_gray, frame_gray # previous frame, new frame
        # take last trajectory stored -> convert to 32 bit floating point number
        p0 = np.float32([trajectory[-1] for trajectory in trajectories]).reshape(-1, 1, 2) 
        p1, _st, _err = cv.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params) # return new point
        p0r, _st, _err = cv.calcOpticalFlowPyrLK(img1, img0, p1, None, **lk_params)
        d = abs(p0 - p0r).reshape(-1, 2).max(-1) # calculate differnece between these 2 points 

        # defined criteria that states it is good optical flow if the difference is less than 1:
        good = d < 1

        # initialize list of new trajectories
        new_trajectories = []

        # get all the trajectories:
        for trajectory, (x,y), good_flag in zip(trajectories, p1.reshape(-1, 2), good):
            # skip it if it is not defined as good optical flow based off the difference 
            if not good_flag:
                continue
            # otherwise... append the x, y coordinates to the trajectory
            trajectory.append((x,y))
            # check if it has now exceeded the trajectopry length, and if so, delete the beginning of the trajectory
            if len(trajectory) > trajectory_len:
                del trajectory[0]
            # add this new trajectory to the new_trajectories list
            new_trajectories.append(trajectory)
            # newest detected point:
            cv.circle(frame_color, (int(x), int(y)), 2, (0,0,255), -1)


        # once this has been done for all trajectories --> update the trajectories variables with the new ones
        trajectories = new_trajectories

        # drawing the trajectories:
        cv.polylines(frame_color, [np.int32(trajectory) for trajectory in trajectories], False, color=(255, 0, 0)) 
        cv.putText(frame_color, 'track count: %d' %len(trajectories), (20, 50), cv.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(0, 0, 255)) # color=red


    # UPDATE INTERVAL - when to update & detect new features 
    if frame_idx % detect_interval == 0:
        mask = np.zeros_like(frame_gray)
        mask[:] = 255

        # take the latest point in the latest trajectory
        # display the latest point as a mask and use it to detect corners when updating features we want to track
        for x, y in [np.int32(trajectory[-1]) for trajectory in trajectories]:
            cv.circle(mask, (x,y), 5, 0, -1)


        # detect the good features to track 
        # p = good features we are going to track
        p = cv.goodFeaturesToTrack(frame_gray, mask=mask, **feature_params)
        if p is not None:
            # if good features can be tracked, add that to the trajectories 
            for x, y in np.float32(p).reshape(-1, 2):
                trajectories.append([(x,y)])



    # once the end of the iteration for this frame has been reached... 

    # update to the next frame:
    frame_idx += 1
    # reassign what the previous frame is
    prev_gray = frame_gray

    # end the time:
    end = time.time()
    # calculate FPS:
    fps = 1 / (end-start)

    # show results:
    cv.putText(frame_color, f"{fps:.2f} FPS", (20,30), cv.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(0, 0, 255)) # color = red
    # cv.imshow("Optical Flow", frame_color)
    # cv.imshow("Mask", mask)
    cv.imwrite(f"opticFlow_example/opticFlow_{file_name}.jpg", frame_color)

    # ???
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

# once done...
# capture.release()
cv.destroyAllWindows()

# # used to give the more time for certain processes to complete 
# #################
# time.sleep(5)
# #################



'''
NOTES:

# reading metadata
capture = cv2.VideoCapture('026c7465-309f6d33.mp4')
n_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))


# reading all frames one by one
img_idx = 0 
for frame in range(n_frames):
    ret, img = capture.read()
    if ret == False:
        break
    if frame % 100 == 0:
        axs[img_idx].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[img_idx].set_title(f'Frame: {frame}')
        axs[img_idx].axis('off')
        img_idx += 1
'''


\DEFINING PARAMETERS

\DEFINING FEATURE PARAMETERS

\INITIALIZING VARIABLES FOR TRAJECTORIES


BEFORE THE WHILE LOOP


SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000001.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000002.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000003.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000004.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000005.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000006.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000007.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000008.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000009.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000010.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000011.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000012.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b8-0000013.jpg

SUCCESSFULLY ENTERED WHILE LOOP

01c71072-718028b

"\nNOTES:\n\n# reading metadata\ncapture = cv2.VideoCapture('026c7465-309f6d33.mp4')\nn_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))\n\n\n# reading all frames one by one\nimg_idx = 0 \nfor frame in range(n_frames):\n    ret, img = capture.read()\n    if ret == False:\n        break\n    if frame % 100 == 0:\n        axs[img_idx].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))\n        axs[img_idx].set_title(f'Frame: {frame}')\n        axs[img_idx].axis('off')\n        img_idx += 1\n"

In [ ]:
''' 
*** LOOK INTO KLD TRACKER (~ Lucas Kanade)
--> migh be able to find source code for using KLD tracker 
'''